## TCGA LGG GBM 1p19q codeletion meta calculation

In [1]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import shutil
import tempfile
import sys
import glob
import pdb

import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
import pandas as pd
import scipy
from scipy import ndimage
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.nn.functional as F

from monai.apps import DecathlonDataset
from monai.config import print_config
from monai.data import DataLoader
from monai.losses import DiceLoss
from monai.metrics import DiceMetric
from monai.losses import GeneralizedWassersteinDiceLoss
from monai.networks.nets import UNet, DynUNet
from monai.transforms import (
    Activations,
    AsDiscrete,
    AsChannelFirstd,
    adaptor,
    CenterSpatialCropd,
    Compose,
    AddChanneld,
    LoadImaged,
    MapTransform,
    NormalizeIntensityd,
    ResizeWithPadOrCropd,
    Orientationd,
    RandFlipd,
    RandScaleIntensityd,
    ScaleIntensityRanged,
    ScaleIntensityd,
    SpatialPadd,
    ScaleIntensityRangePercentilesd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    RandAdjustContrastd,
    AdjustContrastd,
    RandSpatialCropd,
    RandZoomd,
    Resized,
    RandRotated,
    Rotated,
    Flipd,
    CropForegroundd,
    Rand3DElasticd,
    EnsureChannelFirstd,
    Spacingd,
    RandGaussianNoised,
    RandGaussianSmoothd,
    CastToTyped,
    ToTensord,
)
from monai.config import KeysCollection
from monai.transforms.compose import MapTransform, Randomizable
from collections.abc import Iterable
from typing import Any, Dict, Hashable, Mapping, Optional, Sequence, Tuple, Union

from monai.utils import set_determinism
from monai.utils import (
    ensure_tuple,
    ensure_tuple_rep,
    ensure_tuple_size,
)

import glob
import monai
import nibabel as nib
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

#from monai.inferers import sliding_window_inference
from monai.networks.layers import Norm
from monai.metrics import compute_meandice
import random
import multiprocessing as mp

### monai and ignite based imports
#import logging
import sys
import sklearn

from tqdm import tqdm
import torch
import torchio as tio
from itkwidgets import view

monai.config.print_config()

MONAI version: 0.9.0
Numpy version: 1.22.3
Pytorch version: 1.10.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: af0e0e9f757558d144b655c63afcea3a4e0a06f5
MONAI __file__: /home/mmiv-ml/anaconda3/envs/sa_tumorseg22/lib/python3.9/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.9
Nibabel version: 4.0.1
scikit-image version: 0.19.3
Pillow version: 9.0.1
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.11.2
tqdm version: 4.64.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.1
pandas version: 1.4.2
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-rec

In [2]:
pd.set_option('display.max_colwidth', None)
data_rpath = '/home/mmiv-ml/data'

In [3]:
BraTS20Subjectsp1q19WithMetaDF  = pd.read_csv('assets/BraTS_TCGA_LGG_GBM_LGG_1p19qDFMetaFew.csv')
BraTS20Subjectsp1q19WithMetaDF

,BraTS2021,t1wPath,t1cwPath,t2wPath,flairPath,segPath,brain_maskPath,brain_mask_ch2Path,brain_mask_ch1Path,BraTS2020,...,Histology,Grade,Data Collection,IDH,1p19q_co_deletion_bin,IDH,1p19q_co_deletion,CV_group,is_merged_2,is_merged_0
0,BraTS2021_00140,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_00140/ROI_BraTS2021_00140.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_00140/BraTS2021_00140_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_00140/BraTS2021_00140_BrainROI.nii.gz,BraTS20_Training_233,...,glioblastoma,G4,HGG,WT,0.0,NaN,non-codel,3.0,both,NaN
1,BraTS2021_01283,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_01283/ROI_BraTS2021_01283.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01283/BraTS2021_01283_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01283/BraTS2021_01283_BrainROI.nii.gz,BraTS20_Training_165,...,glioblastoma,G4,HGG,WT,0.0,NaN,non-codel,2.0,both,NaN
2,BraTS2021_01528,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_01528/ROI_BraTS2021_01528.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01528/BraTS2021_01528_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01528/BraTS2021_01528_BrainROI.nii.gz,BraTS20_Training_323,...,oligoastrocytoma,G2,LGG,Mutant,0.0,IDH1,non-codel,3.0,both,NaN
3,BraTS2021_01503,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_01503/ROI_BraTS2021_01503.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01503/BraTS2021_01503_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01503/BraTS2021_01503_BrainROI.nii.gz,BraTS20_Training_298,...,oligodendroglioma,G3,LGG,Mutant,1.0,IDH1,codel,3.0,both,NaN
4,BraTS2021_01453,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01453/BraTS2021_01453_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01453/BraTS2021_01453_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS20

In [4]:
BraTS20Subjectsp1q19WithMetaDF.columns

Index(['BraTS2021', 't1wPath', 't1cwPath', 't2wPath', 'flairPath', 'segPath',
       'brain_maskPath', 'brain_mask_ch2Path', 'brain_mask_ch1Path',
       'BraTS2020', 'BraTS2019', 'BraTS2018', 'BraTS2017', 'CohortName',
       'SiteNo_Originating_Institution', 'TCGA_ID', 'is_merged_1', 'Age',
       'Gender', 'Histology', 'Grade', 'Data Collection', '   IDH',
       '1p19q_co_deletion_bin', 'IDH', '1p19q_co_deletion', 'CV_group',
       'is_merged_2', 'is_merged_0'],
      dtype='object')

In [5]:
all_files = [{'image': (image_t1cw, image_t2), 'label': label_name, 'brain_mask':brain_mask, 'TCGA_ID':TCGA_ID} 
             for image_t1cw, image_t2, label_name, brain_mask, TCGA_ID \
             in zip(BraTS20Subjectsp1q19WithMetaDF['t1cwPath'], BraTS20Subjectsp1q19WithMetaDF['t2wPath'],\
                    BraTS20Subjectsp1q19WithMetaDF['segPath'], BraTS20Subjectsp1q19WithMetaDF['brain_mask_ch2Path'], BraTS20Subjectsp1q19WithMetaDF['TCGA_ID'])]


#np.random.shuffle(val_infer_files)

afewval_all_files=all_files[10:20]
len(all_files), len(afewval_all_files)
all_files[200:]

[{'image': ('/raid/brats2021/TCGA_LGG_GBM/BraTS-Processed_TCGA_LGG_GBM/Pre-operative_TCGA_LGG_NIfTI_and_Segmentations_BraTS2017TestingData/TCGA-HT-7874/TCGA-HT-7874_1995.09.02_t1Gd.nii.gz',
   '/raid/brats2021/TCGA_LGG_GBM/BraTS-Processed_TCGA_LGG_GBM/Pre-operative_TCGA_LGG_NIfTI_and_Segmentations_BraTS2017TestingData/TCGA-HT-7874/TCGA-HT-7874_1995.09.02_t2.nii.gz'),
  'label': '/raid/brats2021/TCGA_LGG_GBM/5Ensemble_TCGA_LGG_GBM_Infer/TCGA-HT-7874/TCGA-HT-7874.nii.gz',
  'brain_mask': '/raid/brats2021/TCGA_LGG_GBM/T1wxnBrain_ROIs_TCGA_LGG_GBM/TCGA-HT-7874/TCGA-HT-7874_BrainROIT1wx2.nii.gz',
  'TCGA_ID': 'TCGA-HT-7874'},
 {'image': ('/raid/brats2021/TCGA_LGG_GBM/BraTS-Processed_TCGA_LGG_GBM/Pre-operative_TCGA_LGG_NIfTI_and_Segmentations_BraTS2017TestingData/TCGA-DU-6408/TCGA-DU-6408_1986.05.21_t1Gd.nii.gz',
   '/raid/brats2021/TCGA_LGG_GBM/BraTS-Processed_TCGA_LGG_GBM/Pre-operative_TCGA_LGG_NIfTI_and_Segmentations_BraTS2017TestingData/TCGA-DU-6408/TCGA-DU-6408_1986.05.21_t2.nii.gz'),
 

In [6]:
class ConvertToMultiChannelBased4Calculation(MapTransform):
    
    """
     GD-enhancing tumor (ET — label 4), 
     the peritumoral edema (ED — label 2), and 
     the necrotic and non-enhancing tumor core (NCR/NET — label 1)

    """
    
    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            result = []
            
            d[key]=np.squeeze(d[key], axis = 0) # Converting 1, H, W, D to H, W, D

            # merge labels 1, 2 and 4 to construct WT
            result.append(
                np.logical_or(
                    np.logical_or(d[key] == 2, d[key] == 4), d[key] == 1
                )
            )
            # merge label 1 and label 4 to construct TC
            result.append(np.logical_or(d[key] == 1, d[key] == 4))
            # label 4 is ET
            result.append(d[key] == 4)
            # label 2 edema
            result.append(d[key] == 2)
            # label 1 necrosis
            result.append(d[key] == 1)
            
            d[key] = np.stack(result, axis=0).astype(np.int16)
        return d
    

In [7]:
keys = ["image", "label"]

volcalc_transform = Compose([
    LoadImaged(keys=keys, reader = "NibabelReader"),
    EnsureChannelFirstd(keys=keys),
    ConvertToMultiChannelBased4Calculation(keys = 'label'),
    Orientationd(keys=keys, axcodes="LPS"),    
])

vol_all_ds = monai.data.Dataset(data = all_files, transform = volcalc_transform)
#vol_train_ds = monai.data.CacheDataset(data = train_files, transform = volcalc_transform, cache_rate = 1.0)

wt = list()
wt_x, wt_y, wt_z = list(), list(), list()
tc = list()
tc_x, tc_y, tc_z = list(), list(), list()
et = list()
et_x, et_y, et_z = list(), list(), list()
ed = list()
ed_x, ed_y, ed_z = list(), list(), list()
nec = list()
nec_x, nec_y, nec_z = list(), list(), list()
adict = {}
fnameIDList = list()

for k in tqdm(range(len(vol_all_ds))):
    asubDict = vol_all_ds[k]
    asub = asubDict['label']
    
    #fnameIDList.append(os.path.basename(asubDict['label_meta_dict']['filename_or_obj'])[:-len('_seg.nii.gz')])
    #pdb.set_trace()
    fnameIDList.append(asubDict['TCGA_ID'])
    
   
    
    n_chnl = asub.shape[0]
    a_vol = list()
    
    wt.append(np.sum(asub[0]==1))
    com =  ndimage.measurements.center_of_mass(asub[0])
    wt_x.append(com[0])
    wt_y.append(com[1])
    wt_z.append(com[2])
    
    tc.append(np.sum(asub[1]==1))
    com =  ndimage.measurements.center_of_mass(asub[1])
    tc_x.append(com[0])
    tc_y.append(com[1])
    tc_z.append(com[2])
    
    et.append(np.sum(asub[2]==1))
    com =  ndimage.measurements.center_of_mass(asub[2])
    et_x.append(com[0])
    et_y.append(com[1])
    et_z.append(com[2])
    
    ed.append(np.sum(asub[3]==1))
    com =  ndimage.measurements.center_of_mass(asub[3])
    ed_x.append(com[0])
    ed_y.append(com[1])
    ed_z.append(com[2])
    
    
    nec.append(np.sum(asub[4]==1))
    com =  ndimage.measurements.center_of_mass(asub[4])
    nec_x.append(com[0])
    nec_y.append(com[1])
    nec_z.append(com[2])
    
adict = {'TCGA_ID':fnameIDList, 'WTVol':wt, 'TCVol':tc, 'ETVol':et, 'EDVol':ed, 'NECVol':nec, 'WT_CoordX':wt_x, 'WT_CoordY':wt_y, 'WT_CoordZ':wt_z,\
        'TC_CoordX':tc_x, 'TC_CoordY':tc_y, 'TC_CoordZ':tc_z, 'ET_CoordX':et_x, 'ET_CoordY':et_y, 'ET_CoordZ':et_z,
         'ED_CoordX':ed_x, 'ED_CoordY':ed_y, 'ED_CoordZ':ed_z, 'NEC_CoordX':nec_x, 'NEC_CoordY':nec_y, 'NEC_CoordZ':nec_z}
    
    
BraTS21MetaDf = pd.DataFrame.from_dict(adict) 
display(BraTS21MetaDf)      

100%|██████████████████████████████████████████████████████████████████████| 368/368 [07:29<00:00,  1.22s/it]


,TCGA_ID,WTVol,TCVol,ETVol,EDVol,NECVol,WT_CoordX,WT_CoordY,WT_CoordZ,TC_CoordX,...,TC_CoordZ,ET_CoordX,ET_CoordY,ET_CoordZ,ED_CoordX,ED_CoordY,ED_CoordZ,NEC_CoordX,NEC_CoordY,NEC_CoordZ
0,TCGA-12-3650,16748,2702,2642,14046,60,163.367089,172.468593,86.216324,168.671725,...,79.846040,168.685087,167.653671,79.886450,162.346647,173.396768,87.441763,168.083333,167.200000,78.066667
1,TCGA-02-0046,143759,39933,35721,103826,4212,150.311076,143.524155,69.415445,145.667718,...,59.537701,145.484701,134.678620,59.585174,152.096980,146.947874,73.214571,147.219848,134.146249,59.135090
2,TCGA-HT-7473,232098,104710,4674,127388,100036,84.313781,138.598846,65.267736,71.958543,...,63.167014,77.531023,144.899230,82.371416,94.469503,140.150948,66.994481,71.698179,136.327992,62.269723
3,TCGA-DU-7300,45690,14507,94,31183,14413,106.469359,77.105012,76.322478,105.135038,...,76.952368,110.542553,73.074468,70.808511,107.090113,82.676138,76.029439,105.099771,65.077985,76.992437
4,TCGA-06-0646,23456,5415,5096,18041,319,82.768503,121.368648,65.455150,85.894367,...,67.801662,86.031397,128.011381,67.940149,81.830275,119.381631,64.750845,83.705329,127.626959,65.589342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,LGG-651,34391,0,0,34391,0,150.500422,115.393242,68.151900,NaN,...,NaN,NaN,NaN,NaN,150.500422,115.393242,68.151900,NaN,NaN,NaN
364,LGG-658,30449,209,0,30240,209,136.229466,166.687149,107.381983,140.095694,...,97.712919,NaN,NaN,NaN,136.202745,166.640807,107.448810,140.095694,173.392344,97.712919
365,LGG-659,12216,3,0,12213,3,131.580223,105.936559,122.249427,130.666667,...,123.000000,NaN,NaN,NaN,131.580447,105.934087,122.249243,130.666667,116.000000,123.000000
366,LGG-660,30799,4195,0,26604,4195,88.225949,187.050002,73.777688,88.338498,...,76.130393,NaN,NaN,NaN,88.208202,185.948429,73.406706,88.338498,194.035995,76.130393


In [9]:
#BraTS21MetaDf.to_csv('assets/BraTS21MetaDf.csv', index=False, header = True)

In [8]:
all_files[209]

{'image': ('/raid/brats2021/LGG_1p19q_rawNifti/LGG_1p19q_BraTSLikeProcess_mnibet/LGG-104/LGG-104_t1Gd.nii.gz',
  '/raid/brats2021/LGG_1p19q_rawNifti/LGG_1p19q_BraTSLikeProcess_mnibet/LGG-104/LGG-104_t2.nii.gz'),
 'label': '/raid/brats2021/LGG_1p19q_rawNifti/4Ensemble_LGG_1p19q_Infer/LGG-104/LGG-104_pred.nii.gz',
 'brain_mask': '/raid/brats2021/LGG_1p19q_rawNifti/T1wx2Brain_ROIs_LGG_1p19q/LGG-104/LGG-104_BrainROIT1cwx2.nii.gz',
 'TCGA_ID': 'LGG-104'}

In [9]:
BraTS20Subjectsp1q19WithMetaDF

,BraTS2021,t1wPath,t1cwPath,t2wPath,flairPath,segPath,brain_maskPath,brain_mask_ch2Path,brain_mask_ch1Path,BraTS2020,...,Histology,Grade,Data Collection,IDH,1p19q_co_deletion_bin,IDH,1p19q_co_deletion,CV_group,is_merged_2,is_merged_0
0,BraTS2021_00140,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_00140/ROI_BraTS2021_00140.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_00140/BraTS2021_00140_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_00140/BraTS2021_00140_BrainROI.nii.gz,BraTS20_Training_233,...,glioblastoma,G4,HGG,WT,0.0,NaN,non-codel,3.0,both,NaN
1,BraTS2021_01283,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_01283/ROI_BraTS2021_01283.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01283/BraTS2021_01283_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01283/BraTS2021_01283_BrainROI.nii.gz,BraTS20_Training_165,...,glioblastoma,G4,HGG,WT,0.0,NaN,non-codel,2.0,both,NaN
2,BraTS2021_01528,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_01528/ROI_BraTS2021_01528.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01528/BraTS2021_01528_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01528/BraTS2021_01528_BrainROI.nii.gz,BraTS20_Training_323,...,oligoastrocytoma,G2,LGG,Mutant,0.0,IDH1,non-codel,3.0,both,NaN
3,BraTS2021_01503,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_01503/ROI_BraTS2021_01503.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01503/BraTS2021_01503_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01503/BraTS2021_01503_BrainROI.nii.gz,BraTS20_Training_298,...,oligodendroglioma,G3,LGG,Mutant,1.0,IDH1,codel,3.0,both,NaN
4,BraTS2021_01453,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01453/BraTS2021_01453_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01453/BraTS2021_01453_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS20

In [10]:
BraTS20Subjectsp1q19WithMetaDF_1 = BraTS20Subjectsp1q19WithMetaDF.merge(BraTS21MetaDf, how='inner', on  = 'TCGA_ID', indicator = 'is_merged_3', validate = 'one_to_one') ##copy = True by default
BraTS20Subjectsp1q19WithMetaDF_1

,BraTS2021,t1wPath,t1cwPath,t2wPath,flairPath,segPath,brain_maskPath,brain_mask_ch2Path,brain_mask_ch1Path,BraTS2020,...,ET_CoordX,ET_CoordY,ET_CoordZ,ED_CoordX,ED_CoordY,ED_CoordZ,NEC_CoordX,NEC_CoordY,NEC_CoordZ,is_merged_3
0,BraTS2021_00140,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00140/BraTS2021_00140_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_00140/ROI_BraTS2021_00140.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_00140/BraTS2021_00140_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_00140/BraTS2021_00140_BrainROI.nii.gz,BraTS20_Training_233,...,168.685087,167.653671,79.886450,162.346647,173.396768,87.441763,168.083333,167.200000,78.066667,both
1,BraTS2021_01283,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01283/BraTS2021_01283_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_01283/ROI_BraTS2021_01283.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01283/BraTS2021_01283_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01283/BraTS2021_01283_BrainROI.nii.gz,BraTS20_Training_165,...,145.484701,134.678620,59.585174,152.096980,146.947874,73.214571,147.219848,134.146249,59.135090,both
2,BraTS2021_01528,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01528/BraTS2021_01528_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_01528/ROI_BraTS2021_01528.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01528/BraTS2021_01528_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01528/BraTS2021_01528_BrainROI.nii.gz,BraTS20_Training_323,...,77.531023,144.899230,82.371416,94.469503,140.150948,66.994481,71.698179,136.327992,62.269723,both
3,BraTS2021_01503,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01503/BraTS2021_01503_seg.nii.gz,/raid/brats2021/T1wx4Brain_ROIs/BraTS2021_01503/ROI_BraTS2021_01503.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01503/BraTS2021_01503_BrainROIT1cwx2.nii.gz,/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training/BraTS2021_01503/BraTS2021_01503_BrainROI.nii.gz,BraTS20_Training_298,...,110.542553,73.074468,70.808511,107.090113,82.676138,76.029439,105.099771,65.077985,76.992437,both
4,BraTS2021_01453,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01453/BraTS2021_01453_t1.nii.gz,/rai

In [13]:
(BraTS20Subjectsp1q19WithMetaDF_1['is_merged_3']=='both').sum()

368

In [14]:
BraTS20Subjectsp1q19WithMetaDF_1.to_csv('assets/BraTS_TCGA_LGG_GBM_LGG_1p19qDFMoreMeta.csv', index = False, header = True)

In [16]:
BraTS20Subjectsp1q19WithMetaDF_1.index.values

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18